In [1]:
import pandas as pd
from google.colab import drive
from collections import Counter
drive.mount('/content/drive')

file_path = '/content/drive/MyDrive/AI_Ch_ИИван/rbc_news_annotated_exact_tags (1) - rbc_news_annotated_exact_tags (1).csv'

df = pd.read_csv(file_path)
df.head()

Mounted at /content/drive


,Заголовок,Ссылка,Дата,Текст новости,Теги
0,NVIDIA обошла Apple и стала самой дорогой комп...,https://www.rbc.ru/quote/news/article/672b468a...,2024-11-06T15:21:25+03:00,Капитализация производителя чипов NVIDIA прев...,"NVIDIA, Apple, Microsoft"
1,Акции Tesla прибавили 15% после слов Дональда ...,https://www.rbc.ru/quote/news/article/672b3102...,2024-11-06T13:39:00+03:00,Акции компании Илона Маска Tesla подскочили на...,Tesla
2,Акции медиакомпании Дональда Трампа подскочили...,https://www.rbc.ru/quote/news/article/672b21e4...,2024-11-06T12:19:09+03:00,"Акции Trump Media & Technology Group (DJT), ко...","Курс доллара ЦБ, Фьючерс на доллар/рубль (USDR..."
3,Курс юаня к рублю снизился на Мосбирже на фоне...,https://www.rbc.ru/quote/news/article/672afaf0...,2024-11-06T10:26:03+03:00,"Курс юаня к рублю CNY/RUB снижается на 0,55%, ...","Индекс МосБиржи, Индекс РТС, Индекс МосБиржи в..."
4,Индекс Мосбиржи вырос на 3% на ожиданиях побед...,https://www.rbc.ru/quote/news/article/672a13f8...,2024-11-06T10:12:04+03:00,Индекс Мосбиржи (IMOEX) вырос на открытии торг...,"Индекс МосБиржи, Индекс РТС, Индекс МосБиржи в..."


In [ ]:
# Задание основных и родственных тегов для каждой компании
broad_related_tags_for_companies = {
    "Яндекс": ["HeadHunter", "Ozon", "VK", "Mail.Ru Group", "Рамблер", "Тинькофф", "Ситимобил"],
    "VK": ["Яндекс", "Ozon", "HeadHunter", "Mail.Ru Group", "Одноклассники", "AliExpress", "Delivery Club"],
    "Сбербанк": ["ВТБ", "ТКС Холдинг", "Росбанк", "Альфа-Банк", "Тинькофф", "Совкомбанк", "Росгосстрах"],
    "Северсталь": ["НЛМК", "Русал", "ММК", "Норникель", "Металлоинвест", "ЧТПЗ", "ТМК", "Интерпайп", "Северсталь Логистика", "Металлургия", "Инфраструктура"],
    "ГК «Самолет»": ["ПИК СЗ", "ГК «Эталон»", "Группа ЛСР", "ДОНСТРОЙ", "Интеко", "РостНедвижимость", "РосСтройИнвест", "Центр Девелопмент", "Жилстрой", "Девелопмент", "Недвижимость"],
    "МТС": ["Мегафон", "Билайн", "Tele2", "Ростелеком", "Яндекс.Такси", "SkyLink", "Дом.ru", "ЭР-Телеком", "Т2 Мобайл", "Связь", "Телекоммуникации"],
    "Газпром": ["ЛУКОЙЛ", "Роснефть", "НОВАТЭК", "Газпром нефть", "Башнефть", "Транснефть", "Роснефтегаз"],
    "Русагро": ["Черкизово", "АФГ Националь", "МИРАТОРГ", "Агрокомплекс им. Н. Ткачева", "Эконива", "Брянская мясная компания", "Агроинвест", "Молвест", "Сельское хозяйство", "Пищевая промышленность"]
}

# Функция для отбора ровно 3 новостей для каждой компании из одной группы
def select_news_within_batch(df, company, related_tags, target_count=3):
    # Сначала ищем точные совпадения
    exact_matches = df[df['Теги'].str.contains(f"\\b{company}\\b", case=False, na=False)]
    selected_news = exact_matches.head(target_count)

    # Если точных совпадений меньше нужного, добавляем новости с родственными тегами
    if len(selected_news) < target_count:
        remaining_count = target_count - len(selected_news)
        related_matches = df[df['Теги'].apply(lambda x: any(tag in x for tag in related_tags))]
        related_matches = related_matches[~related_matches.index.isin(selected_news.index)]
        selected_news = pd.concat([selected_news, related_matches.head(remaining_count)])

    # Ограничиваем итог до ровно 3 записей
    return selected_news.head(target_count)

# Инициализация списка для хранения всех отобранных новостей
all_selected_news = []

# Обработка каждой группы по 100 новостей
for part in df_parts:
    for company, related_tags in broad_related_tags_for_companies.items():
        # Отбор ровно 3 новостей для текущей компании в текущей группе
        selected_news = select_news_within_batch(part, company, related_tags, target_count=3)

        # Добавляем информацию о компании и сохраняем в общий список
        selected_news['Компания'] = company
        all_selected_news.append(selected_news)

# Создание единого DataFrame с отобранными новостями
final_selected_news_df = pd.concat(all_selected_news, ignore_index=True)

# Просмотр итогового DataFrame
final_selected_news_df

# Сохранение итогового DataFrame в CSV
final_selected_news_df.to_csv("selected_news_for_all_companies.csv", index=False)

In [ ]:
import pandas as pd
import nltk
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Загрузка данных
file_path = "selected_news_for_all_companies.csv"
df = pd.read_csv(file_path)

# Токенизация текста на предложения и слова
nltk.download('punkt')
def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
    return tokenized_sentences, sentences

# Обучение модели Word2Vec на всём наборе данных
all_sentences = []
for text in df['Текст новости']:
    tokenized_sentences, _ = preprocess_text(str(text))
    all_sentences.extend(tokenized_sentences)

word2vec_model = Word2Vec(sentences=all_sentences, vector_size=100, window=5, min_count=2, workers=4)

# Функция для получения вектора предложения
def sentence_vector(sentence, model):
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

# Функция для получения трех самых важных предложений из новости
def get_top_important_sentences(text, model, top_n=3):
    tokenized_sentences, original_sentences = preprocess_text(text)

    # Векторизация предложений
    sentence_vectors = [sentence_vector(sentence, model) for sentence in tokenized_sentences]

    # Вектор всего текста
    text_vector = np.mean([vec for vec in sentence_vectors if vec is not None], axis=0)

    # Рассчитываем косинусное сходство предложений с вектором всего текста
    similarities = cosine_similarity([text_vector], sentence_vectors)[0]

    # Находим индексы топ-N предложений
    important_indices = similarities.argsort()[-top_n:][::-1]
    important_sentences = [original_sentences[i] for i in important_indices]

    return important_sentences

# Разделение данных на 4 части и выборка по 3 отрывка на компанию из каждой части
df['Группа'] = df.groupby(df.index // 100).ngroup()  # Добавляем столбец группы
result_data = []

for group_num in range(4):  # Для каждой группы из 100 новостей
    group_df = df[df['Группа'] == group_num]
    for company in group_df['Компания'].unique():  # Для каждой компании
        company_news = group_df[group_df['Компания'] == company]
        if not company_news.empty:
            # Берем первую новость для компании и получаем три важных предложения
            important_sentences = get_top_important_sentences(company_news.iloc[0]['Текст новости'], word2vec_model, top_n=3)
            # Сохраняем каждое из 3 предложений как отдельную запись
            for sentence_num, sentence in enumerate(important_sentences, start=1):
                result_data.append({
                    'Группа': group_num,
                    'Компания': company,
                    'Отрывок номер': sentence_num,
                    'Важный отрывок': sentence
                })

# Создание итогового DataFrame с отрывками
result_df = pd.DataFrame(result_data)

# Сохранение в CSV
result_df.to_csv("important_sentences_separated.csv", index=False)
print(result_df)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


    Группа      Компания  Отрывок номер  \
0        0        Яндекс              1   
1        0        Яндекс              2   
2        0        Яндекс              3   
3        0            VK              1   
4        0            VK              2   
5        0            VK              3   
6        0      Сбербанк              1   
7        0      Сбербанк              2   
8        0      Сбербанк              3   
9        0    Северсталь              1   
10       0    Северсталь              2   
11       0    Северсталь              3   
12       0       Газпром              1   
13       0       Газпром              2   
14       0       Газпром              3   
15       0       Русагро              1   
16       0       Русагро              2   
17       0       Русагро              3   
18       0           МТС              1   
19       0           МТС              2   
20       0           МТС              3   
21       0  ГК «Самолет»              1   
22       0 

In [13]:
print(df.columns)


Index(['Заголовок', 'Ссылка', 'Дата', 'Текст новости', 'Теги'], dtype='object')


In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка предобработанных эмбеддингов
embeddings_df = pd.read_pickle("/content/news_embeddings.pkl")  # Загружаем эмбеддинги новостей

# Функция для выбора релевантных новостей и отрывков
def select_top_news_and_snippets(company_name, top_k=3):
    # Фильтрация новостей на основе наличия компании в столбце 'Теги'
    relevant_news = embeddings_df[embeddings_df['Теги'].str.contains(company_name, case=False, na=False)].head(top_k)

    # Для каждой новости выбираем три самых релевантных отрывка
    results = []
    for _, news in relevant_news.iterrows():
        # Вектор всей новости (усредненный по предложениям)
        text_embedding = np.mean(news['Эмбеддинги предложений'], axis=0)

        # Вычисляем косинусное сходство для каждого предложения с текстом
        similarities = cosine_similarity(news['Эмбеддинги предложений'], text_embedding.reshape(1, -1)).flatten()
        top_indices = similarities.argsort()[-3:][::-1]  # Три самых релевантных предложения
        top_snippets = [news['Предложения'][i] for i in top_indices]

        # Сохраняем заголовок, отрывки и ссылку
        results.append({
            "Заголовок": news['Заголовок'],
            "Ссылка": news['Ссылка'],
            "Отрывки": top_snippets
        })
    return results

# Пример использования функции
company_name = "Яндекс"  # Замените на нужную компанию
news_results = select_top_news_and_snippets(company_name)

# Вывод результатов
for news in news_results:
    print(f"Заголовок: {news['Заголовок']}")
    print(f"Ссылка: {news['Ссылка']}")
    for i, snippet in enumerate(news['Отрывки'], start=1):
        print(f"Отрывок {i}: {snippet}")
    print("\n")


Заголовок: Акции бывшей материнской компании «Яндекса» упали на NASDAQ на 25%
Ссылка: https://www.rbc.ru/quote/news/article/671630c09a79478530578c32
Отрывок 1: «Отчетность охватывает совсем небольшой период и возможно привлечение дополнительного финансирования или выкуп акций», — считают они.
Отрывок 2: закрыла сделку по продаже своего российского бизнеса консорциуму инвесторов и полностью вышла из состава акционеров российской группы.
Отрывок 3: Аркадий Волож в июне 2022 года подпал под санкции Евросоюза, после чего покинул все должности в основанном им «Яндексе».


Заголовок: NASDAQ возобновит торги акциями бывшей материнской компании «Яндекса»
Ссылка: https://www.rbc.ru/quote/news/article/6711fd3d9a7947196692717f
Отрывок 1: После закрытия сделки сооснователь «Яндекса» Аркадий Волож вернулся к управлению нидерландской компанией.
Отрывок 2: Штаб-квартира компании и ее основные центры исследований и разработок находятся в Амстердаме.
Отрывок 3: В июле завершился процесс делистинга  акц

In [19]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка предобработанных эмбеддингов
embeddings_df = pd.read_pickle("/content/news_embeddings.pkl")  # Загружаем эмбеддинги новостей

# Список 8 компаний
companies = ["Яндекс", "VK", "Сбербанк", "Северсталь", "ГК «Самолет»", "МТС", "Газпром", "Русагро"]

# Функция для выбора релевантных новостей и отрывков
def select_top_news_and_snippets(company_name, top_k=3):
    # Фильтрация новостей по наличию названия компании в 'Теги'
    relevant_news = embeddings_df[embeddings_df['Теги'].str.contains(company_name, case=False, na=False)].head(top_k)

    # Для каждой новости выбираем три самых релевантных отрывка
    results = []
    for _, news in relevant_news.iterrows():
        # Вектор всей новости (усредненный по предложениям)
        text_embedding = np.mean(news['Эмбеддинги предложений'], axis=0)

        # Вычисляем косинусное сходство для каждого предложения с текстом
        similarities = cosine_similarity(news['Эмбеддинги предложений'], text_embedding.reshape(1, -1)).flatten()
        top_indices = similarities.argsort()[-3:][::-1]  # Три самых релевантных предложения
        top_snippets = [news['Предложения'][i] for i in top_indices]

        # Сохраняем заголовок, отрывки и ссылку
        results.append({
            "Заголовок": news['Заголовок'],
            "Ссылка": news['Ссылка'],
            "Отрывки": top_snippets
        })
    return results

# Вывод результатов для каждой из 8 компаний
for company_name in companies:
    print(f"Компания: {company_name}")
    news_results = select_top_news_and_snippets(company_name)
    if news_results:
        for news in news_results:
            print(f"Заголовок: {news['Заголовок']}")
            print(f"Ссылка: {news['Ссылка']}")
            for i, snippet in enumerate(news['Отрывки'], start=1):
                print(f"Отрывок {i}: {snippet}")
            print("\n")
    else:
        prin


Компания: Яндекс
Заголовок: Акции бывшей материнской компании «Яндекса» упали на NASDAQ на 25%
Ссылка: https://www.rbc.ru/quote/news/article/671630c09a79478530578c32
Отрывок 1: «Отчетность охватывает совсем небольшой период и возможно привлечение дополнительного финансирования или выкуп акций», — считают они.
Отрывок 2: закрыла сделку по продаже своего российского бизнеса консорциуму инвесторов и полностью вышла из состава акционеров российской группы.
Отрывок 3: Аркадий Волож в июне 2022 года подпал под санкции Евросоюза, после чего покинул все должности в основанном им «Яндексе».


Заголовок: NASDAQ возобновит торги акциями бывшей материнской компании «Яндекса»
Ссылка: https://www.rbc.ru/quote/news/article/6711fd3d9a7947196692717f
Отрывок 1: После закрытия сделки сооснователь «Яндекса» Аркадий Волож вернулся к управлению нидерландской компанией.
Отрывок 2: Штаб-квартира компании и ее основные центры исследований и разработок находятся в Амстердаме.
Отрывок 3: В июле завершился процес

In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка предобработанных эмбеддингов
embeddings_df = pd.read_pickle("/content/news_embeddings.pkl")  # Загружаем эмбеддинги новостей

# Список 8 компаний
companies = ["Яндекс", "VK", "Сбербанк", "Северсталь", "ГК «Самолет»", "МТС", "Газпром", "Русагро"]

# Функция для выбора релевантных новостей и отрывков
def select_top_news_and_snippets(company_name, top_k=3, snippet_length=2):
    # Фильтрация новостей по наличию названия компании в 'Теги'
    relevant_news = embeddings_df[embeddings_df['Теги'].str.contains(company_name, case=False, na=False)].head(top_k)

    # Для каждой новости выбираем три самых релевантных отрывка
    results = []
    for _, news in relevant_news.iterrows():
        # Вектор всей новости (усредненный по предложениям)
        text_embedding = np.mean(news['Эмбеддинги предложений'], axis=0)

        # Вычисляем косинусное сходство для каждого предложения с текстом
        similarities = cosine_similarity(news['Эмбеддинги предложений'], text_embedding.reshape(1, -1)).flatten()
        top_indices = similarities.argsort()[-3:][::-1]  # Индексы 3 самых релевантных предложений

        # Формирование более длинных отрывков
        top_snippets = []
        for idx in top_indices:
            # Выбираем `snippet_length` предложений, начиная с самого релевантного
            snippet = " ".join(news['Предложения'][idx:idx + snippet_length])
            top_snippets.append(snippet)

        # Сохраняем заголовок, отрывки и ссылку
        results.append({
            "Заголовок": news['Заголовок'],
            "Ссылка": news['Ссылка'],
            "Отрывки": top_snippets
        })
    return results

# Вывод результатов для каждой из 8 компаний
for company_name in companies:
    print(f"Компания: {company_name}")
    news_results = select_top_news_and_snippets(company_name)
    if news_results:
        for news in news_results:
            print(f"Заголовок: {news['Заголовок']}")
            print(f"Ссылка: {news['Ссылка']}")
            for i, snippet in enumerate(news['Отрывки'], start=1):
                print(f"Отрывок {i}: {snippet}")
            print("\n")
    else:
        print("Нет релевантных новостей.\n")


Компания: Яндекс
Заголовок: Акции бывшей материнской компании «Яндекса» упали на NASDAQ на 25%
Ссылка: https://www.rbc.ru/quote/news/article/671630c09a79478530578c32
Отрывок 1: «Отчетность охватывает совсем небольшой период и возможно привлечение дополнительного финансирования или выкуп акций», — считают они. В июле нидерландская компания Yandex N.V.
Отрывок 2: закрыла сделку по продаже своего российского бизнеса консорциуму инвесторов и полностью вышла из состава акционеров российской группы. Общая сумма сделки составила $5,4 млрд.
Отрывок 3: Аркадий Волож в июне 2022 года подпал под санкции Евросоюза, после чего покинул все должности в основанном им «Яндексе». 13 марта этого года Евросоюз исключил его из черных списков.


Заголовок: NASDAQ возобновит торги акциями бывшей материнской компании «Яндекса»
Ссылка: https://www.rbc.ru/quote/news/article/6711fd3d9a7947196692717f
Отрывок 1: После закрытия сделки сооснователь «Яндекса» Аркадий Волож вернулся к управлению нидерландской компание